<a href="https://colab.research.google.com/github/winterglade/ml/blob/main/Team_Python%60%D1%87%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Код адаптированный под Google Collab

In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import files

# Загрузка данных
data_path = '/content/dataset.csv'
data = pd.read_csv(data_path)

# Удаление пропусков
data = data.dropna()

# Преобразование категориальных данных
le = LabelEncoder()
for col in ['device_type', 'tran_code', 'card_type', 'oper_type', 'card_status']:
    data[col] = le.fit_transform(data[col].astype(str))

# Создание временных признаков
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
data['hour'] = data['datetime'].dt.hour
data['day_of_week'] = data['datetime'].dt.dayofweek

# Нормализация числовых данных
scaler = MinMaxScaler()
data[['sum', 'balance', 'pin_inc_count']] = scaler.fit_transform(data[['sum', 'balance', 'pin_inc_count']])

# Признаки для модели
features = ['sum', 'balance', 'pin_inc_count', 'device_type', 'hour',
            'day_of_week', 'tran_code', 'oper_type', 'card_status']

X = data[features]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Определение модели KNN для поиска аномалий
knn = NearestNeighbors()

# Параметры для оптимизации
param_grid = {
    'n_neighbors': [3, 5, 10, 15, 20, 30, 50, 70, 100],  # Количество ближайших соседей
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'cosine', 'minkowski'],  # Метрики расстояния
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Алгоритмы поиска
    'leaf_size': [10, 20, 30, 40, 50, 70, 100],  # Размер листа в деревьях
    'p': [1, 2, 3],  # Параметр для метрики Minkowski
    'n_jobs': [-1]  # Использовать все процессоры
}

# Подбор гиперпараметров через GridSearchCV
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=3, scoring='f1', verbose=1, n_jobs=-1)

# Обучение модели с подбором параметров
grid_search.fit(X_train)

# Лучшие параметры
print("Лучшие параметры KNN:", grid_search.best_params_)

# Получаем лучшую модель
best_knn = grid_search.best_estimator_

# Расстояния до ближайших соседей для обучения
distances, _ = best_knn.kneighbors(X_train)

# Задание порога для аномалий
threshold = 0.8 * distances.max()

# Создание меток для обучения (True - аномалия, False - нормальная точка)
y_train = (distances[:, -1] > threshold)
y_test = (best_knn.kneighbors(X_test)[0][:, -1] > threshold)

# Определение моделей
rf = RandomForestClassifier(random_state=42)
lr = LogisticRegression(max_iter=1000, random_state=42)

# Создание VotingClassifier (мягкое голосование)
voting_clf = VotingClassifier(estimators=[
    ('rf', rf),
    ('lr', lr)
], voting='soft')

# Обучение на тренировочных данных с метками
voting_clf.fit(X_train, y_train)

# Получение предсказаний
y_train_pred = voting_clf.predict(X_train)
y_test_pred = voting_clf.predict(X_test)

# Оценка метрик
print(f"F1-Score (Train): {f1_score(y_train, y_train_pred)}")
print(f"F1-Score (Test): {f1_score(y_test, y_test_pred)}")
print(f"Classification Report (Test):\n{classification_report(y_test, y_test_pred)}")

# Визуализация 1: Аномалии по сумме и балансу
data['anomaly_flag'] = voting_clf.predict(X[features])  # Метки аномалий для всех данных
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data, x='sum', y='balance', hue='anomaly_flag', palette={True: 'red', False: 'blue'})
plt.title("Аномалии по сумме и балансу")
plt.xlabel("Сумма")
plt.ylabel("Баланс")
plt.legend(title='Аномалия', loc='upper right')
plt.show()

# Визуализация 2: Корреляционная матрица
plt.figure(figsize=(12, 8))
sns.heatmap(data[features + ['anomaly_flag']].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Корреляция признаков с аномалиями")
plt.show()

# Визуализация 3: Плотность точек для аномалий и нормальных данных
plt.figure(figsize=(12, 8))
sns.kdeplot(x=data[data['anomaly_flag'] == True]['sum'], y=data[data['anomaly_flag'] == True]['balance'],
            color='red', label='Аномалии', fill=True, alpha=0.5)
sns.kdeplot(x=data[data['anomaly_flag'] == False]['sum'], y=data[data['anomaly_flag'] == False]['balance'],
            color='blue', label='Нормальные', fill=True, alpha=0.5)
plt.title("Плотность аномалий и нормальных данных")
plt.xlabel("Сумма")
plt.ylabel("Баланс")
plt.legend()
plt.show()

# Вывод статистики по аномалиям
print("Количество аномальных операций:")
print(data['anomaly_flag'].value_counts())

# Сохранение результатов в CSV файл
output_path = '/content/preds_with_optimal_knn.csv'
data[['anomaly_flag']].to_csv(output_path, index=False)

# Скачивание файла
files.download(output_path)